# Organisation structurelle et géographique du terrorisme mondial


Charlotte Gallezot, Hilaire Perchenet et Gauthier Guinet


In [1149]:
#Importation des libraires utiles

import pandas as pd
import numpy as np
import math 

#Importation de la GTD

data = pd.read_csv('/Users/gauthierguinet/Desktop/GGG/Data/data.csv',sep = ';')

#Structure de la data

data.head(3)


/Users/gauthierguinet/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (4,6,31,33,61,62,63,70,76,79,90,92,94,96,101,107,112,114,115,117,118,119,121) have mixed types. Specify dtype option on import or set low_memory=False.



,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [520]:

####################################################################################################################
################################################Cleaning of the data################################################
####################################################################################################################
#1)Removing of several useless columns
   
#ex: data=data.drop(columns=['dbsource'])
    
#Date
data=data.drop(columns=['approxdate',
                        'resolution'])

#location
data=data.drop(columns=['extended',
                        'vicinity',
                        'specificity',
                        'location'])



'''
#Weapons
data=data.drop(columns=['weaptype1',
                        'weaptype1_txt',
                        'weapsubtype1',
                        'weapsubtype1_txt',
                        'weaptype2',
                        'weaptype2_txt',
                        'weapsubtype2',
                        'weapsubtype2_txt',
                        'weaptype3',
                        'weaptype3_txt',
                        'weapsubtype3',
                        'weapsubtype3_txt',
                        'weaptype4',
                        'weaptype4_txt',
                        'weapsubtype4',
                        'weapsubtype4_txt',
                        'weapdetail',
                        'suicide',
                        'individual',
                        'nperpcap'])
'''                        
                        

#Claim
data=data.drop(columns=['compclaim',
                        'claimmode',
                        'claimmode_txt',
                        'claim2',
                        'claimmode2',
                        'claimmode2_txt',
                        'claim3',
                        'claimmode3',
                        'claimmode3_txt'])

#US citizens
data=data.drop(columns=['nkillus',
                        'nwoundus'])

#Material Dammage
data=data.drop(columns=['property',
                        'propextent',
                        'propextent_txt',
                        'propvalue',
                        'propcomment'])


'''
#Kidnapping
data=data.drop(columns=['ishostkid',
                        'nhostkid',
                        'nhostkidus',
                        'nhours',
                        'ndays',
                        'divert',
                        'kidhijcountry',
                        'ransom',
                        'ransomamt',
                        'ransomamtus',
                        'ransompaid',
                        'ransompaidus',
                        'ransomnote',
                        'hostkidoutcome',
                        'hostkidoutcome_txt',
                        'nreleased'])
'''

#Additionnal information
data=data.drop(columns=['addnotes'])

#Sources
data=data.drop(columns=['scite1',
                        'scite2',
                        'scite3',
                        'dbsource'])

#2)We deleted the terrorist attacks whose autor is Unknown:
data=data.ix[data['gname'] != 'Unknown']
  
    
#3) For severals columns, the int were considered as string and written 8,5 for exemple
    
data['nwound']=data['nwound'].astype(str)
data['nwound']=[x.replace(',', '.') for x in data['nwound']]
data['nwound']=[x.replace('nan', '0') for x in data['nwound']]
data['nwound']=[x.replace('.0', '') for x in data['nwound']]
data['nwound']=[x.replace('.5', '') for x in data['nwound']]
data['nwound']=data['nwound'].astype(int)
    
data['nkill']=data['nkill'].fillna(0).astype(int)
                    
#3)We deleted the attacks whose affiliation with a terrorist organisation is not certain

#Suggestion: We might take a look in a second time at all the kidnapping/hijackind data


/Users/gauthierguinet/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [996]:


####################################################################################################################
############################################Modification of the data################################################
####################################################################################################################

#1) We added a columns to quantify the severity of the attack

data['nwound']=data['nwound'].astype(str)
data['nwound']=[x.replace(',', '.') for x in data['nwound']]
data['nwound']=[x.replace('nan', '0') for x in data['nwound']]
data['nwound']=[x.replace('.0', '') for x in data['nwound']]
data['nwound']=[x.replace('.5', '') for x in data['nwound']]
data['nwound']=data['nwound'].astype(int)
    
data['nkill']=data['nkill'].fillna(0).astype(int)
    
data['nkill']=data['nkill'].fillna(0).astype(int)
data['Severity']= (3*data['nkill']+data['nwound'])



In [523]:
###################################################################################################################
############################################# Terrorist group df ##################################################
###################################################################################################################

#We look now for the descending order of the most "dangerous" terrorist groups 

#All the terrorists groups
group=data['gname'].unique()
n=len(group)


#Initialisation
terrorist_group = pd.DataFrame(0, index = group,columns=['Lethality','Number of attacks',
                                                         'International Attacks LOG',
                                                         'International Attacks IDEO',
                                                         'International Ratio LOG',
                                                         'International Ratio IDEO'
                                                         ])

for i in range(n):
    terrorist_group.iloc[i,0]=data[data['gname']==terrorist_group.iloc[i].name]['Severity'].sum()
    
terrorist_group=terrorist_group.sort_values(by = 'Lethality', ascending = False)  


###############################  We create severals features for each terrorist group #############################

#Number of attacks

df_nb_attacks = data['gname'].value_counts()

for group_name in group:
    
    terrorist_group.loc[group_name,'Number of attacks']=df_nb_attacks.loc[group_name]
    

#International attacks 

#Disserter sur le caractère international dans le choix qu'on a montré


#LOGISTIC INTERNATIONAL ATTACKS

df_nb_inter_attacks_log = data[data['INT_LOG']==1]['gname'].value_counts()

for group_name in df_nb_inter_attacks_log.index:
    
    terrorist_group.loc[group_name,'International Attacks LOG']=df_nb_inter_attacks_log.loc[group_name]
    

#IDEOLOGICAL INTERNATIONAL ATTACKS 
    
df_nb_inter_attacks_ideo = data[data['INT_IDEO']==1]['gname'].value_counts()

for group_name in df_nb_inter_attacks_ideo.index:
    
    terrorist_group.loc[group_name,'International Attacks IDEO']=df_nb_inter_attacks_ideo.loc[group_name]    
    
#International ratio 

terrorist_group['International Ratio LOG']=terrorist_group['International Attacks LOG']/terrorist_group['Number of attacks']

terrorist_group['International Ratio IDEO']=terrorist_group['International Attacks IDEO']/terrorist_group['Number of attacks']

In [524]:
###################################################################################################################
############################################# Terrorist group df ##################################################
###################################################################################################################


#De la première DataFrame, on en déduit une seconde décrivant non plus les évènements mais 
#les organisations terroristes 


#####################################  Localisation of the attacks of the group  ##################################

#Definition of the regions

region_localisation = ['North_America_loc','Central_America_loc','South_America_loc',
                       'East_Asia_loc','Southeast_Asia_loc','South_Asia_loc','Central_Asia_loc',
                       'Western_Europe_loc','Eastern_Europe_loc','Middle_East_&_North_Africa_loc',
                       'Sub_Saharan_Africa_loc','Australasia _&_Oceania_loc']

dict_regions= {1:'North_America_loc',2:'Central_America_loc',3:'South_America_loc',
              4:'East_Asia_loc',5:'Southeast_Asia_loc',6:'South_Asia_loc',7:'Central_Asia_loc',
              8:'Western_Europe_loc',9:'Eastern_Europe_loc',10:'Middle_East_&_North_Africa_loc',
              11:'Sub_Saharan_Africa_loc',12:'Australasia _&_Oceania_loc'}

#Initialisation

for region in region_localisation:
    terrorist_group[region]=0

    
    
#Implémentation des features    
    
for i in data.index:
    terrorist_group.loc[data.loc[i,'gname'],dict_regions[data.loc[i,'region']]]+=1


In [525]:
#### Renormalisation des variables de localisation

for region in region_localisation:
    terrorist_group[region]=terrorist_group[region]/terrorist_group['Number of attacks']


In [526]:
# Ajout de la Sévérité Moyenne

terrorist_group['Mean Severity']=terrorist_group['Lethality']/terrorist_group['Number of attacks']

In [527]:
#Ajout de l'Attack type de prédilection 

terrorist_group['Favorite Attack Type']=0

for group_name in group:
    
    terrorist_group.loc[group_name,'Favorite Attack Type'] = data[data['gname']==group_name]['attacktype1'].value_counts().index[0]

In [528]:
#Ajout du Target Type de prédilection 

terrorist_group['Favorite Target Type']=0

for group_name in group:
    
    terrorist_group.loc[group_name,'Favorite Target Type'] = data[data['gname']==group_name]['targtype1'].value_counts().index[0]

In [530]:
#Ajout du Weapon Type de prédilection 

terrorist_group['Favorite Weapon Type']=0

for group_name in group:
    
    terrorist_group.loc[group_name,'Favorite Weapon Type'] = data[data['gname']==group_name]['weaptype1'].value_counts().index[0]

In [535]:
#Ajout du Ratio Claim of Responsibility

terrorist_group['Ratio Claim of Responsibility']=0

df_claim = data[data['claimed']==1]['gname'].value_counts()

for group_name in df_claim.index:
    
    terrorist_group.loc[group_name,'Ratio Claim of Responsibility']=df_claim.loc[group_name] 
    
terrorist_group['Ratio Claim of Responsibility']=terrorist_group['Ratio Claim of Responsibility']/terrorist_group['Number of attacks']

In [537]:
#Ajout du Ratio Kidnapping

terrorist_group['Ratio Kidnapping']=0

df_kid = data[data['ishostkid']==1]['gname'].value_counts()

for group_name in df_kid.index:
    
    terrorist_group.loc[group_name,'Ratio Kidnapping']=df_kid.loc[group_name] 
    
terrorist_group['Ratio Kidnapping']=terrorist_group['Ratio Kidnapping']/terrorist_group['Number of attacks']

In [538]:
#Structure de la dataframe terrorist_group

terrorist_group.head(10)

,Lethality,Number of attacks,International Attacks LOG,International Attacks IDEO,International Ratio LOG,International Ratio IDEO,North_America_loc,Central_America_loc,South_America_loc,East_Asia_loc,...,Eastern_Europe_loc,Middle_East_&_North_Africa_loc,Sub_Saharan_Africa_loc,Australasia _&_Oceania_loc,Mean Severity,Favorite Attack Type,Favorite Target Type,Favorite Weapon Type,Ratio Claim of Responsibility,Ratio Kidnapping
Islamic State of Iraq and the Levant (ISIL),147441,5613,193,5613,0.034384,1.000000,0.000000,0.0,0.00000,0.0,...,0.000713,0.991092,0.000178,0.000178,26.267771,3,14,6,0.419205,0.114199
Taliban,116162,7478,55,1004,0.007355,0.134260,0.000000,0.0,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,15.533833,3,3,6,0.482348,0.106178
Boko Haram,70456,2418,331,357,0.136890,0.147643,0.000000,0.0,0.00000,0.0,...,0.000000,0.000000,1.000000,0.000000,29.138131,2,14,6,0.098428,0.084367
Liberation Tigers of Tamil Eelam (LTTE),43998,1606,10,125,0.006227,0.077833,0.000000,0.0,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,27.396015,3,4,5,0.021793,0.031756
Shining Path (SL),37834,4555,14,136,0.003074,0.029857,0.000220,0.0,0.99978,0.0,...,0.000000,0.000000,0.000000,0.000000,8.306037,3,14,6,0.002415,0.034907
Al-Qaida,36092,74,0,74,0.000000,1.000000,0.054054,0.0,0.00000,0.0,...,0.000000,0.135135,0.067568,0.000000,487.729730,3,19,6,0.418919,0.094595
Al-Shabaab,35614,3288,418,1175,0.127129,0.357360,0.000000,0.0,0.00000,0.0,...,0.000000,0.000000,1.000000,0.000000,10.831509,3,4,6,0.378954,0.110401
Farabundo Marti National Liberation Front (FMLN),28260,3351,21,52,0.006267,0.015518,0.000000,1.0,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,8.433303,2,4,5,0.000000,0.049239
Tehrik-i-Taliban Pakistan (TTP),27658,1351,16,97,0.011843,0.071799,0.000740,0.0,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,20.472243,3,14,6,0.549223,0.119171
Al-Qaida in Iraq,23486,638,6,36,0.009404,0.056426,0.000000,0.0,0.00000,0.0,...,0.000000,0.998433,0.000000,0.000000,36.811912,3,14,6,0.648903,0.040752


In [676]:
#####################################################################################################################
###################################### Terrorist Group Proximity ####################################################
#####################################################################################################################


#On définit une fonction 3 fonctions proximity entre différents groupes terroristes 
# afin de trouver une possible proximité
# Plus la valeur renvoyée sera elevée, plus les groupes seront proches

#La première traitera de l'impact (lethality, severity,mean severity)
#La seconde traitera des caractéristiques géographiques 
#La troisième traitera des caractéristiques des modes d'action des groupes
#La difficulté est de définir des coefficients pour les différents facteurs entrant en compte
#dans la définition de la distance


def proximity_impact(group1,group2,df):
    
    #1er Bloc: Lethality, Number of Attacks, Mean Severity, varie entre 0 et 3
    
    prox=0
    
    
    #Lethality, varie entre 0 et 1
    
    prox +=min(df.loc[group1,'Lethality'],df.loc[group2,'Lethality'])/max(df.loc[group1,'Lethality'],df.loc[group2,'Lethality'])
    
    #Number of attacks, varie entre 0 et 1
    
    prox +=min(df.loc[group1,'Number of attacks'],df.loc[group2,'Number of attacks'])/max(df.loc[group1,'Number of attacks'],df.loc[group2,'Number of attacks'])
    
    #Mean Severity, varie entre 0 et 1
    
    prox +=min(df.loc[group1,'Mean Severity'],df.loc[group2,'Mean Severity'])/max(df.loc[group1,'Mean Severity'],df.loc[group2,'Mean Severity'])
                                                                            
    return prox 


def proximity_local(group1,group2,df):

    prox = 0
    
    #2ème Bloc: International ratio and Localisation, varie entre 0 et 3
    
    
    #International Ratio LOG
    
    prox += 1-abs(df.loc[group1,'International Ratio LOG']-df.loc[group2,'International Ratio LOG'])
    
    #Localisation
                      
    for region in region_localisation:
            if not(df.loc[group1,region]==0 or df.loc[group2,region]==0):
                prox += math.exp(-abs(df.loc[group1,region]-df.loc[group2,region]))*(df.loc[group1,region]+df.loc[group2,region])*5
            
                           
    return prox/3


def proximity_mode(group1,group2,df):
    
    prox = 0

    
    #3ème bloc: 'Favorite Attack Type','Favorite Target Type','International Ratio IDEO',
    #'Ratio Claim of Responsibility','Ratio Kidnapping','Favorite Weapon Type'
    
                           
    #Favorite Attack Type, varie entre 0 et 1
                           
                           
    if (df.loc[group1,'Favorite Attack Type']==df.loc[group2,'Favorite Attack Type']):
        prox+=1
                           

    #Favorite Target Type, varie entre 0 et 1
                           
                           
    if (df.loc[group1,'Favorite Target Type']==df.loc[group2,'Favorite Target Type']):
        prox+=1          
        
    #Favorite Weapon Type, varie entre 0 et 1
                           
                           
    if (df.loc[group1,'Favorite Weapon Type']==df.loc[group2,'Favorite Target Type']):
        prox+=1     
        
    #International Ratio IDEO
    
    prox += 1-abs(df.loc[group1,'International Ratio IDEO']-df.loc[group2,'International Ratio IDEO'])
    
    #Ratio Kidnapping
    
    prox += 1-abs(df.loc[group1,'Ratio Kidnapping']-df.loc[group2,'Ratio Kidnapping'])
     
    #Ratio Claim of Responsibility
    
    prox += 1-abs(df.loc[group1,'Ratio Claim of Responsibility']-df.loc[group2,'Ratio Claim of Responsibility'])
    
    return prox/2 



def df_proxy(n):
    
    #Choix des groupes terrorists à étudier
    groupes_cibles = terrorist_group.head(n).index
    
    #Initisalisation des variables
    proximity_terrorist_local=pd.DataFrame(0,index=groupes_cibles,columns=groupes_cibles)
    proximity_terrorist_impact=pd.DataFrame(0,index=groupes_cibles,columns=groupes_cibles)
    proximity_terrorist_mode=pd.DataFrame(0,index=groupes_cibles,columns=groupes_cibles)



    for group1 in groupes_cibles:
        for group2 in groupes_cibles:
            if (group1 != group2):
                proximity_terrorist_local.loc[group2,group1]=proximity_local(group2,group1,terrorist_group)
                proximity_terrorist_impact.loc[group2,group1]=proximity_impact(group2,group1,terrorist_group)
                proximity_terrorist_mode.loc[group2,group1]=proximity_mode(group2,group1,terrorist_group)
                
    return proximity_terrorist_impact,proximity_terrorist_local,proximity_terrorist_mode

In [353]:
########################################## Sauvegarde en CSV #######################################################

terrorist_group.to_csv('/Users/gauthierguinet/Desktop/GGG/terrorism_group.csv',sep = ',')

nom_groupe.to_csv('/Users/gauthierguinet/Desktop/GGG/name_terrorism_group.csv',sep = ',')

In [809]:
#Fonction créant un fichier csv à partir des données afin de l'utiliser dans Gephi

def for_gephi(n,limite,Title='',Nodes=False):
    
    
    
    #Initialisation des proximités
    
    proximity_terrorist_impact,proximity_terrorist_local,proximity_terrorist_mode = df_proxy(n)
                
    proximity_terrorist=proximity_terrorist_mode+proximity_terrorist_local+proximity_terrorist_impact
    
    
    #Initialisation des edges
    link_terrorist_group=pd.DataFrame(0,index=range(0),columns=['Source','Target','Weight'])
    name_groupe = proximity_terrorist.index

    indice = 0 
    for i in range(n):
        for j in range(i+1,n):
            dist=proximity_terrorist.loc[name_groupe[i],name_groupe[j]]
            if dist>limite:
                link_terrorist_group.loc[indice]=0
                link_terrorist_group.loc[indice,'Source']=name_groupe[i]
                link_terrorist_group.loc[indice,'Target']=name_groupe[j]
                link_terrorist_group.loc[indice,'Weight']=dist
                indice +=1
    
    if Nodes:
        #Initiation des nodes
        nom_groupe=pd.DataFrame(proximity_terrorist.index,columns=['id'])
        label_groupe=pd.DataFrame(proximity_terrorist.index,columns=['label'])
    
        nodes_groupe=pd.concat([nom_groupe,label_groupe],axis=1) 
    
    
    ####enregistrement pour gephi 
    nodes_groupe.to_csv('/Users/gauthierguinet/Desktop/GGG/Data/Gephi/Nodes_'
                            +str(n)+'_'+Title+'.csv',
                            sep = ',',index=False)
    
    link_terrorist_group.to_csv('/Users/gauthierguinet/Desktop/GGG/Data/Gephi/Edges_'
                                +str(n)+'_'+Title+'.csv',
                                sep = ',',index=False)

In [811]:
#Appel à la fonction ci-dessus

for_gephi(300,3.5,'all',Nodes=True)

In [696]:
#Calcul de caractéristiques de la distance ainsi définie

proximity_terrorist_impact,proximity_terrorist_local,proximity_terrorist_mode = df_proxy(50)

proximity_terrorist=proximity_terrorist_impact+proximity_terrorist_local+proximity_terrorist_mode

#We compare the distances 

comparaison = pd.DataFrame(0,index=['impact','local','mode','all'],
                           columns=['mean', 'std', '25%', '50%', '75%', 'max'])


describe_impact = proximity_terrorist_impact.describe()
describe_local = proximity_terrorist_local.describe()
describe_mode = proximity_terrorist_mode.describe()
describe_all = proximity_terrorist.describe()


for compa in comparaison.columns:
    comparaison.loc['impact',compa]=describe_impact.loc[compa].mean()
    comparaison.loc['local',compa]=describe_local.loc[compa].mean()
    comparaison.loc['mode',compa]=describe_mode.loc[compa].mean()
    comparaison.loc['all',compa]=describe_all.loc[compa].mean()
    
comparaison

,mean,std,25%,50%,75%,max
impact,1.254454,0.534610,0.886977,1.233231,1.588127,2.527708
local,0.944098,1.082479,0.335751,0.402101,1.285979,3.524807
mode,1.515463,0.444267,1.241445,1.551275,1.794673,2.344263
all,3.714015,1.415486,2.825945,3.426395,4.405507,7.256518


In [ ]:
####################################################################################################################
################################################ DATA INTERPRETATION ###################################################
###################################################################################################################

#On va maintenant afficher géographiquement les villes des assauts des clusters de terroristes

In [1135]:
###Data after gephi treatement

###The centrality & others features correspond to before the intra-class treatement 

terrorist_cluster = pd.read_csv('/Users/gauthierguinet/Desktop/GGG/Data/terrorist_cluster.csv',sep = ',')


countries_geo = pd.read_excel('/Users/gauthierguinet/Desktop/GGG/Data/Gephi/countries.xlsx',sep = ';')



In [820]:
### We group the terrorist following their cluster 

terrorist_group_0 = terrorist_cluster['Id'][terrorist_cluster['modularity_class']==0]
terrorist_group_1 = terrorist_cluster['Id'][terrorist_cluster['modularity_class']==1]
terrorist_group_2 = terrorist_cluster['Id'][terrorist_cluster['modularity_class']==2]
terrorist_group_3 = terrorist_cluster['Id'][terrorist_cluster['modularity_class']==3]

In [1150]:
# For the following data, we have treated the intra-class in order to make
# the features more relevants
class_0 = pd.read_csv('/Users/gauthierguinet/Desktop/GGG/Data/Gephi/terrorist_cluster_class0.csv',
                                       sep = ',')
class_0 = class_0[class_0['modularity_class']==0]

#Exemple des données extraites après traitement par gephi
class_0.sort_values(by ='Authority',ascending=False).head(10)

,Id,Label,timeset,weighted degree,modularity_class,Eccentricity,closnesscentrality,harmonicclosnesscentrality,betweenesscentrality,Authority,Hub,pageranks,clustering,triangles,eigencentrality
203,Hadramawt Province of the Islamic State,Hadramawt Province of the Islamic State,NaN,170.820062,0,4,0.454545,0.576667,107.328723,0.257543,0.257543,0.023606,0.513834,130,1.000000
126,Jaysh al-Islam (Syria),Jaysh al-Islam (Syria),NaN,163.329453,0,4,0.493421,0.597778,266.656944,0.252014,0.252014,0.023105,0.506494,117,0.983832
211,Ahrar al-Sham,Ahrar al-Sham,NaN,158.795314,0,5,0.471698,0.581556,109.635058,0.248657,0.248657,0.021745,0.561905,118,0.972449
128,Islamic Front (Syria),Islamic Front (Syria),NaN,155.893004,0,4,0.487013,0.587778,204.391496,0.248390,0.248390,0.021778,0.538095,113,0.970400
255,Mahdi Army,Mahdi Army,NaN,162.666895,0,5,0.465839,0.582667,196.125743,0.243939,0.243939,0.022814,0.476190,110,0.956197
172,Al-Naqshabandiya Army,Al-Naqshabandiya Army,NaN,143.861497,0,5,0.465839,0.568222,66.211442,0.239271,0.239271,0.019633,0.643275,110,0.933592
244,Military Council of the Tribal Revolutionaries...,Military Council of the Tribal Revolutionaries...,NaN,136.113563,0,4,0.468750,0.561111,91.946806,0.234231,0.234231,0.018542,0.660131,101,0.910396
168,Islamic State in Egypt,Islamic State in Egypt,NaN,133.776380,0,4,0.441176,0.543333,33.085500,0.229536,0.229536,0.018518,0.660131,101,0.886129
122,Hay'at Tahrir al-Sham,Hay'at Tahrir al-Sham,NaN,143.209967,0,5,0.451807,0.559333,85.970005,0.225279,0.225279,0.020419,0.584795,100,0.880782
221,Ansar al-Islam,Ansar al-Islam,NaN,127.895881,0,5,0.428571,0.535333,25.071021,0.220517,0.220517,0.017487,0.691176,94,0.856002


In [1129]:
#Lors des cellules suivantes, on s'attachera à créer une troisième DataFrame
#traitant maintenant des pays en lien avec les différents clusters des 
#groupes terroristes 

#Exemple avec le cluster 2
event_terrorist=data[data['gname'].isin(list(terrorist_group_2))]


#Dataframe des pays associés au cluster 2 ainsi que leurs paramètres 
country_terrorist = pd.DataFrame(0,
                                   index=event_terrorist['country_txt'].unique(),
                                   columns=['id',
                                            'label',
                                            'Region',
                                            'Latitude',
                                            'Longitude',
                                            'Number of attacks',
                                            'Lethality',
                                            'Group Presence',
                                            'Mean year of Attacks',
                                            'Pondered Mean Year',
                                            'Pondered Importance of the Country']
                                    )
    
for country in country_terrorist.index:
    attacks_country = event_terrorist[event_terrorist['country_txt']==country]
    country_terrorist.loc[country,'Region']= attacks_country['region_txt'].iloc[0]
    country_terrorist.loc[country,'Number of attacks']=len(attacks_country)
    country_terrorist.loc[country,'Lethality']=attacks_country['Severity'].sum()
    country_terrorist.loc[country,'Group Presence']=len(attacks_country['gname'].unique())
    country_terrorist.loc[country,'Mean year of Attacks']=attacks_country['iyear'].mean()
    country_terrorist.loc[country,'Pondered Mean Year']=(attacks_country['iyear']*attacks_country['Severity']).sum()
                                                         

        
        
country_terrorist['Pondered Mean Year']=country_terrorist['Pondered Mean Year']\
                                        *(1/country_terrorist['Lethality'])




#Nettoyage des données afin de pouvoir les combiner avec une autre base: country.csv

def data_cleaning_country(country1,country2):
    #we add the data of the old country (1) to the new one (2)
    if country1 in country_terrorist.index:
        if country2 in country_terrorist.index:
            country_terrorist.loc[country1]=country_terrorist.loc[country2]+\
                                     country_terrorist.loc[country1]
        else:
            country_terrorist.loc[country2]=country_terrorist.loc[country1]
        country_terrorist.drop(index=[country1],inplace=True)    

## DATA CLEANING: pour certains noms de pays n'existant plus, nous les regroupons (ex: West Germany)
# d'autres modifications ont été faites dans la database des pays

        
data_cleaning_country('West Germany (FRG)','Germany')
data_cleaning_country('East Germany (GDR)','Germany')
data_cleaning_country('Zaire','Democratic Republic of the Congo')
data_cleaning_country('Rhodesia','Zimbabwe')
data_cleaning_country('South Yemen','Yemen')
data_cleaning_country('South Yemen','Yemen')
data_cleaning_country('Yugoslavia','Bosnia-Herzegovina')
data_cleaning_country('South Sudan','Sudan')
data_cleaning_country('People\'s Republic of the Congo','Republic of the Congo')        
data_cleaning_country('International','Yemen')  
data_cleaning_country('Czechoslovakia','Bosnia-Herzegovina')   



#On ajouter latitude et longitude aux pays, champs initiés à 0 

for country in country_terrorist.index:
    country_terrorist.loc[country,'Latitude']=countries_geo[countries_geo['Name']==country]['Latitude'].iloc[0]
    country_terrorist.loc[country,'Longitude']=countries_geo[countries_geo['Name']==country]['Longitude'].iloc[0]

    
    
country_terrorist=country_terrorist.sort_values(by = 'Lethality', ascending = False) 


In [1125]:
#On exporte la dataframe des nodes pour gephi

country_terrorist['id']=country_terrorist.index
country_terrorist['label']=country_terrorist.index
country_terrorist.to_csv('/Users/gauthierguinet/Desktop/GGG/Data/Gephi/country/Nodes_group3.csv',
                         sep = ',',index=False)

In [1131]:
# On affiche maintenant les liens entre les différents groupes, c'est à dire
#les edges pour Gephi

link_countries = pd.DataFrame(0,index=country_terrorist.index,columns=terrorist_group_2.values)

event_terrorist=data[data['gname'].isin(list(terrorist_group_2))]


#On fait un dict de pays pour gérer les pays renomés
dict_country={'West Germany (FRG)':'Germany',
              'East Germany (GDR)':'Germany',
              'Zaire':'Democratic Republic of the Congo',
              'Rhodesia':'Zimbabwe',
              'South Yemen':'Yemen',
              'South Yemen':'Yemen',
              'Yugoslavia':'Bosnia-Herzegovina',
              'South Sudan':'Sudan',
              'People\'s Republic of the Congo':'Republic of the Congo',
              'International':'Yemen'
             }

#On y ajoute les autres pays
for country in country_terrorist.index:
    dict_country[country]=country

#Pour chaque groupe, on note les nombre d'attaques par pays 

for country in country_terrorist.index:
    attacks_country = event_terrorist[event_terrorist['country_txt']==country]
    for event_id in attacks_country.index:
            link_countries.loc[dict_country[country],attacks_country.loc[event_id,'gname']]+=1
           

link_countries

,Shining Path (SL),Farabundo Marti National Liberation Front (FMLN),Revolutionary Armed Forces of Colombia (FARC),Nicaraguan Democratic Force (FDN),Irish Republican Army (IRA),National Liberation Army of Colombia (ELN),Democratic Revolutionary Alliance (ARDE),Basque Fatherland and Freedom (ETA),M-19 (Movement of April 19),Death Squad,...,Real Irish Republican Army (RIRA),Black September,First of October Antifascist Resistance Group (GRAPO),Red Brigades,Right-Wing Paramilitaries,Revolutionary Armed Forces of Nicaragua (FARN),Institutional Revolutionary Party (PRI),Left-Wing Terrorists,Irish Republican Extremists,Ex-Somoza National Guard
Colombia,4,0,2468,1,0,1529,1,0,543,171,...,0,0,0,0,17,0,0,1,0,0
Peru,4541,0,4,0,0,0,0,0,1,6,...,0,0,0,0,0,0,0,1,0,0
Nicaragua,0,2,0,886,0,0,139,0,1,0,...,0,0,0,0,0,6,0,0,0,21
El Salvador,0,3330,0,0,0,0,0,0,0,58,...,0,0,0,0,0,0,0,17,0,0
United Kingdom,0,0,0,0,2575,0,0,0,0,0,...,38,10,0,1,0,0,0,0,94,0
Spain,0,0,0,0,0,0,0,1975,0,0,...,0,2,207,0,0,0,0,0,0,0
Guatemala,0,5,0,0,0,0,0,0,0,69,...,0,0,0,0,0,0,0,8,0,0
United States,0,0,0,0,2,0,0,0,0,0,...,0,10,0,0,0,0,0,0,0,0
Chad,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Italy,0,0,0,0,0,0,0,15,0,0,...,0,5,0,215,0,0,0,3,0,0


In [1127]:
################################# EDGES Version 1  ###################################

#Pour le moment, il n'y a aucune notion de force entre les liens en fonction des
#autres attaques du groupes, c'est assez intuitif

#dataframe auxiliaire
link_created=pd.DataFrame(-1,index=country_terrorist.index,columns=country_terrorist.index)


#dataframe des edges recherchée

edges_terrorist_group = pd.DataFrame(0,
                                     index=range(0),
                                     columns=['Source','Target','Weight']
                                     
                                     
                                     
                                     
# On parcourt la dataframe des pays attaqués par groupe
#afin de déterminer la force des liens
                                     
indice = 0
for group in link_countries.columns:
    #listes de pays attaqués par le groupe
    attacked_country_group = link_countries[group][link_countries[group]>0]
    n = len(attacked_country_group)
    for i in range(n):
        for j in range(i+1,n):
            #Rang de la Dataframe correspondant au lien 
            rang=link_created.loc[attacked_country_group.index[i],attacked_country_group.index[j]]
            #Si le lien n'existe pas encore en les deux pays, on l'initialise
            if rang==-1:
                rang=indice
                edges_terrorist_group.loc[rang]=0
                edges_terrorist_group.loc[rang,'Source']=attacked_country_group.index[i]
                edges_terrorist_group.loc[rang,'Target']=attacked_country_group.index[j]
                link_created.loc[attacked_country_group.index[i],attacked_country_group.index[j]]=rang
            #on incrémente le lien entre les deux pays
            edges_terrorist_group.loc[rang,'Weight']=edges_terrorist_group.loc[rang,'Weight']+1
            indice +=1



In [1128]:
#On exporte pour Gephi

edges_terrorist_group.to_csv('/Users/gauthierguinet/Desktop/GGG/Data/Gephi/country/Edges_group3.csv',
                             sep = ',',index=False)

In [1145]:
################################# EDGES Version 2 plus complète ###################################

#On coefficiente par l'autority du groupe terrorist 

link_created_bis=pd.DataFrame(-1,index=country_terrorist.index,columns=country_terrorist.index)

edges_terrorist_group_bis = pd.DataFrame(0,
                                       index=range(0),
                                       columns=['Source','Target','Weight']
                                       )

indice = 0


for group in link_countries.columns:
    
    #Valeur actualisée de l'authority du groupe terroriste
    autho=10*terrorist_cluster[terrorist_cluster['Id']==group]['eigencentrality'].iloc[0]
    
    #listes de pays attaqués par le groupe
    attacked_country_group = link_countries[group][link_countries[group]>0]
    n = len(attacked_country_group)
    for i in range(n):
        for j in range(i+1,n):
            
            #Rang de la Dataframe correspondant au lien 
            rang=link_created_bis.loc[attacked_country_group.index[i],attacked_country_group.index[j]]
            #Si le lien n'existe pas encore en les deux pays, on l'initialise
            if rang==-1:
                rang=indice
                edges_terrorist_group_bis.loc[rang]=0
                edges_terrorist_group_bis.loc[rang,'Source']=attacked_country_group.index[i]
                edges_terrorist_group_bis.loc[rang,'Target']=attacked_country_group.index[j]
                link_created_bis.loc[attacked_country_group.index[i],attacked_country_group.index[j]]=rang
            #on incrémente le lien entre les deux pays en lien avec l'authority
            edges_terrorist_group_bis.loc[rang,'Weight']=edges_terrorist_group_bis.loc[rang,'Weight']\
                                                     +autho
            
            indice +=1

In [1151]:
#On filtre puis on exporte sur Gephi

edges_terrorist_group_bis=edges_terrorist_group_bis[edges_terrorist_group_bis['Weight']>17]


edges_terrorist_group_bis.to_csv('/Users/gauthierguinet/Desktop/GGG/Data/Gephi/country/V2_Edges_group2.csv',
                                 sep = ',',index=False)

In [956]:
#Pistes d'exploration:

#### OTER DES DONNEES AFIN D'ESSAYER DE LES PREDIRE ? 

#Analyser les données jusqu'à 2013 puis tenter de prédire les comportements globaux à partir de 
#ceux des leaders qui surgissent 

#Autre question intéressante: prédire trouver l'auteur d'une attaque donnée
#